In [53]:
import re
from collections import Counter
import pymorphy2
from tqdm import tqdm
import os
import pandas as pd
import math
import nltk

In [2]:
marked_path = '/home/nst/mount/data/share/yd/popular_science_texts_store/ner_markup/final_markup/'

In [3]:
def slurp(path):
    with open(path, 'r') as file_object:
        return file_object.read()

In [4]:
def extract_ne(text):
    pattern = re.compile(r'&(.*?)!&')
    nes = re.findall(pattern, text)
    lemmas = []
    for ne in nes:
        divided = ne.split()
        lemma = [morph.parse(word)[0].normal_form for word in divided]
        lemma = ' '.join(lemma)
        lemmas.append(lemma)
    return lemmas

In [74]:
def range_ne(texts_path, top_number):
    nes = []
    texts = []
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            marked_text = slurp(input_path)
            texts.append(marked_text)
    for text in tqdm(texts):
        found_ne = extract_ne(text)
        #print(found_ne)
        nes.extend(found_ne)
    ne_dict = Counter(nes)
    ranged_ne = list(sorted(ne_dict, key = lambda x : x[1], reverse=True))
    print(top_number, "most cited scholars are:\n", ranged_ne[:top_number]) 
    return ranged_ne

## Get NE context

In [118]:
def preprocess(texts):
    #pattern = re.compile(r'[\w.-]+')
    clean_texts = []
    for text in tqdm(texts):
        split_text = re.findall(r'[\w.\'-]+', text, flags=re.UNICODE)
        joined_text = ' '.join(split_text)
        clean_texts.append(joined_text)
    preproc_texts = pd.DataFrame(clean_texts, columns=['preprocessed_texts'])
    return preproc_texts

In [145]:
def extract_entities_indices(clean_text, marked_text):
    indices = []
    entity_index_left = None
    entity_index_right = None
    pattern = re.compile(r'&(.*?)!&')
    nes = re.findall(pattern, marked_text)
   # print(nes)
    for name in nes:
        name = name.split()
        if len(name)==1:
            name = ''.join(name)
            try:
                entity_index_left = clean_text.index(name)
            except ValueError:
                print('Name:', name)
                print('Text_marked:', marked_text)
                raise 
            try:
                entity_index_right = clean_text.index(name) + len(name) 
            except ValueError:
                print('Name:', name)
                print('Text_marked:', marked_text)
                raise                             
        else:
            try:
                entity_index_left = clean_text.index(name[0])
            except ValueError:
                print('Name:', name[0])
                print('Text_marked:', marked_text)
                raise 
            try:
                entity_index_right = clean_text.index(name[-1]) + len(name[-1])
            except ValueError:
                print('Name:', name[0])
                print('Text_marked:', marked_text)
                raise 
        index_pair = (entity_index_left, entity_index_right)
        indices.append(index_pair)
    return indices 

In [146]:
def get_context(clean_text, entities_indices, window):
    left = []
    names = []
    right = []
    for index_pair in entities_indices:
        left_index = index_pair[0]
        right_index = index_pair[1] + 1
        name = clean_text[left_index:right_index]
        names.append(name)
        left_context = clean_text[:left_index].split()
        if len(left_context) < window:
            left_context = clean_text[:left_index]
        else:
            left_context = ' '.join(left_context[-window:])
        left.append(left_context)    
        right_context = clean_text[right_index:].split()
        if len(right_context) < window:
            right_context = clean_text[right_index:]
        else:
            right_context = ' '.join(right_context[:window+1])
        right.append(right_context)
    return left, names, right

In [7]:
def get_all_contexts(clean_texts, marked_texts, window):
    indices_list = [extract_entities_indices(clean_text, marked_text)
              for clean_text, marked_text in zip(clean_texts, marked_texts)]
    lefts_list = []
    names_list = []
    rights_list = []
    for text, indices in zip(clean_texts, indices_list):
        left, names, right = get_context(text, indices, window)
        lefts_list.extend(left)
        names_list.extend(names)
        rights_list.extend(right)
    dataframe = pd.DataFrame(lefts_list, columns=['left_context'])
    dataframe['named_entities'] = names_list
    dataframe['right_context'] = rights_list
    return dataframe

## Make Dataframe and get all NE contexts

### Marked & Preprocessed DataFrame

In [92]:
def make_texts_df(texts_path):
    marked_texts = []
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            marked_text = slurp(input_path)
            marked_texts.append(marked_text)
    marked_df = pd.DataFrame(marked_texts, columns=['marked_texts'])
    return marked_df

In [206]:
marked_df = make_texts_df(marked_path)

In [207]:
preprocessed_texts = preprocess(marked_df.marked_texts)

100%|██████████| 164/164 [00:00<00:00, 750.44it/s]


In [208]:
texts_df = marked_df.join(preprocessed_texts) ;

In [97]:
texts_df.preprocessed_texts[2] ;

### Get all contexts from marked texts with window 5

In [209]:
contexts_df = get_all_contexts(texts_df.preprocessed_texts, texts_df.marked_texts, 5)

In [210]:
len(contexts_df)

1903

In [224]:
remove_duplicates = contexts_df.drop_duplicates()

In [225]:
len(remove_duplicates)

1255

In [227]:
remove_duplicates = remove_duplicates.reset_index(drop=True)
remove_duplicates.head()

,left_context,named_entities,right_context
0,Из выступления лауреата Нобелевской премии,Ричарда Фейнмана,в Калифорнийском технологическом институте в 1959
1,проблема была в 1981 году,Гердом Биннигом,и Генрихом Рорером сотрудниками швейцарского о...
2,1981 году Гердом Биннигом и,Генрихом Рорером,сотрудниками швейцарского отделения IBM сконст...
3,сотрудниками IBM Almaden Research Center,Дональдом Эйглером,и Эрхардом Швейцером в 1990 году
4,Research Center Дональдом Эйглером и,Эрхардом Швейцером,в 1990 году заключался в том


In [228]:
remove_duplicates.to_csv('contexts_draft.tsv', sep='\t')

In [205]:
for root, dirs, files in os.walk(marked_path):
    for file_name in files:
        input_path = marked_path + file_name
        marked_text = slurp(input_path)
        if 'Субдискретизация' in marked_text:
            print(file_name)

company_audiomania_blog_251144.txt


In [160]:
'Когда &Гарри Уиттингтон!& и его ученики &Дерек Бриггс!& и &Саймон Конвей Моррис!&'

'Когда &Гарри Уиттингтон!& и его ученики &Дерек Бриггс!& и &Саймон Конвей Моррис!&'

In [2]:
df = pd.read_csv('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/Popular-Science-Texts-Compling-research/ner_markup/contexts_draft.tsv', sep='\t')

## Count most frequent collocations in contexts

In [33]:
def preprocess(texts):
    preproc_texts = []
    pattern = re.compile(r'\w+')
    try:
        for text in tqdm(texts):
            text = text.lower()
            words = ' '.join(re.findall(pattern, text))
            preproc_texts.append(words)
    except AttributeError:
        print('Corrupted:', text)
        preproc_texts.append('')
    return preproc_texts

In [37]:
preprocessed_left = preprocess(df.left_context)
preprocessed_right = preprocess(df.right_context)

100%|██████████| 1216/1216 [00:00<00:00, 93964.03it/s]


In [36]:
df = df.dropna(axis=0, how='any')
len(df)

1216

In [38]:
preprocessed_df = pd.DataFrame(preprocessed_left, columns=['left'])
preprocessed_df['right'] = preprocessed_right
preprocessed_df.head(3)

,left,right
0,из выступления лауреата нобелевской премии,в калифорнийском технологическом институте в 1959
1,проблема была в 1981 году,и генрихом рорером сотрудниками швейцарского о...
2,1981 году гердом биннигом и,сотрудниками швейцарского отделения ibm сконст...


In [44]:
def compile_tokens_list(texts):
    tokens = []
    for text in texts:
        text = text.split()
    tokens.extend(text)
    tokens = ' '.join(tokens)
    return tokens

In [45]:
tokens_left = compile_tokens_list(preprocessed_df.left)
tokens_right = compile_tokens_list(preprocessed_df.right)

In [47]:
def aprior_probability(dictionary):
    s = sum(dictionary.values())
    for word  in dictionary:
        dictionary[word] /= s
   # aprior_dictionary = dict(wiki_freq).get(word)/s
    return dict(dictionary)

In [48]:
def bigram_probability(bigrams):
    count_bigrams = len(bigrams)
    joined_bigrams = [' '.join(bigram) for bigram in bigrams]
    bigrams_dict = Counter(joined_bigrams)
    for bigram in bigrams_dict:
        bigrams_dict[bigram] /= count_bigrams
    return dict(bigrams_dict)

In [49]:
def count_pmi(bigrams, bigrams_prob, words_aprior):
    joined_bigrams = []
    pmis = []
    for bigram in bigrams:
        #print(bigram)
        joined_bigram = ' '.join(bigram)
        pmi = math.log(bigrams_prob.get(joined_bigram)/
                      (words_aprior.get(bigram[0])*words_aprior.get(bigram[1])))
        joined_bigrams.append(joined_bigram)
        pmis.append(pmi)
    counted_pmi = list(zip(joined_bigrams, pmis))
    return counted_pmi

In [51]:
def range_pmi(pmis_list, top_k):
    pmis_dict = dict(pmis_list)
    best_pmis = sorted(pmis_dict, key = lambda x:x[1], reverse = True)
    print(best_pmis[:top_k])
    return best_pmis

In [52]:
def extract_collocations(tokens, top_best):
    freq = Counter(tokens.split())
    bigrams = list(nltk.bigrams(tokens.split()))
    aprior = aprior_probability(freq)
    bigrams_prob = bigram_probability(bigrams)
    pmi = count_pmi(bigrams, bigrams_prob, aprior)
    best = range_pmi(pmi, top_best)
    return best

In [56]:
# top best left 
collocations_left = extract_collocations(tokens_left, 10)

['мы рассказывали', 'совсем недавно', 'недавно мы', 'рассказывали о']


In [57]:
# top best right
collocations_right = extract_collocations(tokens_right, 10)

['истории об', 'архивной находке', 'его истории', 'об архивной', 'и его']
